In [126]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

project_folder = os.getcwd()
for dirname, _, filenames in os.walk(os.path.join(project_folder,'kaggle/input')):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/Users/haiho/ShopeeCodeLeague/ShopeeCodeLeague2020/1_ORDER-BRUSHING/kaggle/input/.DS_Store
/Users/haiho/ShopeeCodeLeague/ShopeeCodeLeague2020/1_ORDER-BRUSHING/kaggle/input/order_brush_order.csv


In [128]:
import pandas as pd #imports Python Data Analysis Library

df = pd.read_csv('/Users/haiho/ShopeeCodeLeague/ShopeeCodeLeague2020/1_ORDER-BRUSHING/kaggle/input/order_brush_order.csv')

In [129]:
df.event_time = pd.to_datetime(df.event_time)

# Submission format

In [270]:
# Submission format
import pandas as pd
import numpy as np

submission_shopid = pd.DataFrame(df.shopid.unique())
submission_userid = pd.DataFrame(np.zeros(shape=submission_shopid.shape))
submission = pd.DataFrame(columns = ['shopid', 'userid'])
submission.shopid = df.shopid.unique()
submission.userid = str(0)
submission.to_csv("submission.csv", header=True, index=False)

In [272]:
submission.head()

,shopid,userid
0,6042309,0
1,104804492,0
2,8715449,0
3,190969466,0
4,2859407,0


# Preprocessing

In [130]:
df = df.sort_values(by=['event_time'])

In [133]:
df['orderdate'] = [d.date() for d in df['event_time']]
df['ordertime'] = [d.time() for d in df['event_time']]
df['orderhour'] = [d.hour for d in df['ordertime']]

In [136]:
df.head()

,orderid,shopid,userid,event_time,orderdate,ordertime,orderhour
150060,31075200506751,6042309,97707522,2019-12-27 00:00:00,2019-12-27,00:00:00,0
25985,31075200506752,104804492,97707522,2019-12-27 00:00:00,2019-12-27,00:00:00,0
81020,31075200506753,8715449,97707522,2019-12-27 00:00:00,2019-12-27,00:00:00,0
146598,31075201870570,190969466,170182475,2019-12-27 00:00:02,2019-12-27,00:00:02,0
80355,31075205798264,2859407,12532131,2019-12-27 00:00:05,2019-12-27,00:00:05,0


In [287]:
a = df.groupby(['orderdate', 'orderhour', 'shopid'])['userid'].value_counts()
suspicious_shops = {}
for index, value in a.iteritems():
    if value > 3:
        suspicious_shopid = index[2]
        suspicious_userid = index[3]
        if suspicious_shopid not in suspicious_shops.keys():
            suspicious_shops[suspicious_shopid]=str(suspicious_userid)
        else:
            suspicious_shops[suspicious_shopid]+="&"+str(suspicious_userid)
print(suspicious_shops)

{147941492: '147478511&163220673&199110270&208806468&58009118&2812894&77645501&78274359&212480041&164098479&50499134&190222721&43652561&4769572&211429582&205951406&213906397&89835507&59282182&189029587&71026438&88732127&86875149&208806468&37434703&50499134&197551102&213526271&52097335&5333888&95619116&7616036&16950820&86051997&213787697&22184&164098479&205583383&86875149&166081737&208806468&52255254&80060796&13487830&205951406&7616036&154598501&52097335&205583383&198218978&3142265&203538523&3142265&9613337&210362283&195350861&2284151&208806468&9613337&3142265&19970394&206794371&88587531&197551102&23857471&52097335&84226482&12282583&58009118&45163392&199110270&190222721&113418397&43652561&3142265&3524519&55372731&1997501&9535411&213162186&203742758&206585991&50499134&101092594&103256866&3142265&48793634&113830512&164098479&205046026&186403294&205435395&107331095&207802408&29815158&104066552&19443866&23961096&52415138&80060796&3766486&208806468&7872248&30279967&55446317&112716544&5225525

In [288]:
for index, value in submission.iterrows():
    if value[0] in suspicious_shops.keys():
        submission.iloc[index,1] = suspicious_shops[value[0]]

In [289]:
submission.head(100)

,shopid,userid
0,6042309,0
1,104804492,0
2,8715449,9753706
3,190969466,0
4,2859407,0
...,...,...
95,33380999,0
96,68973461,0
97,163522251,0
98,32504026,10221040


In [290]:
submission.to_csv("submission.csv", header=True, index=False)

# draft

In [135]:
df.orderdate.unique()

array([datetime.date(2019, 12, 27), datetime.date(2019, 12, 28),
       datetime.date(2019, 12, 29), datetime.date(2019, 12, 30),
       datetime.date(2019, 12, 31)], dtype=object)

In [191]:
df_groupby_orderhour = df.groupby(['orderdate','orderhour'])
df_groupby_orderhour.size()

orderdate   orderhour
2019-12-27  0            2670
            1            1556
            2             802
            3             543
            4             285
                         ... 
2019-12-31  19           1945
            20           2163
            21           2258
            22           2405
            23           2514
Length: 120, dtype: int64

In [212]:
# df.groupby(['col5','col2']).size().reset_index().groupby('col2')[[0]].max()
df_groupby_orderhour_uniqueuserid = df.groupby(['orderhour','userid'])
df_groupby_orderhour_uniqueuserid.head(20)

,orderid,shopid,userid,event_time,orderdate,ordertime,orderhour
150060,31075200506751,6042309,97707522,2019-12-27 00:00:00,2019-12-27,00:00:00,0
25985,31075200506752,104804492,97707522,2019-12-27 00:00:00,2019-12-27,00:00:00,0
81020,31075200506753,8715449,97707522,2019-12-27 00:00:00,2019-12-27,00:00:00,0
146598,31075201870570,190969466,170182475,2019-12-27 00:00:02,2019-12-27,00:00:02,0
80355,31075205798264,2859407,12532131,2019-12-27 00:00:05,2019-12-27,00:00:05,0
...,...,...,...,...,...,...,...
114113,31507183252446,149254894,193333760,2019-12-31 23:59:43,2019-12-31,23:59:43,23
206585,31507187390691,147941492,40258063,2019-12-31 23:59:47,2019-12-31,23:59:47,23
126212,31507191066628,187123853,2338306,2019-12-31 23:59:51,2019-12-31,23:59:51,23
160657,31507191066627,154074176,2338306,2019-12-31 23:59:51,2019-12-31,23:59:51,23


In [172]:
a = df.groupby(['shopid', 'orderdate', 'orderhour', 'userid'])

In [174]:
print(type(unique_userid_in_1hour))

<class 'pandas.core.series.Series'>


In [173]:
unique_userid_in_1hour.head()

orderdate   orderhour
2019-12-27  0            [97707522, 170182475, 12532131, 195903090, 128...
            1            [30280485, 11424169, 51981728, 93910369, 69925...
            2            [169046011, 3110472, 212970579, 175336440, 188...
            3            [180160633, 102794063, 159315857, 200116970, 1...
            4            [214236745, 168156000, 70815930, 22800224, 332...
Name: userid, dtype: object

In [177]:
unique_userid_in_1hour.keys()

MultiIndex([('2019-12-27',  0),
            ('2019-12-27',  1),
            ('2019-12-27',  2),
            ('2019-12-27',  3),
            ('2019-12-27',  4),
            ('2019-12-27',  5),
            ('2019-12-27',  6),
            ('2019-12-27',  7),
            ('2019-12-27',  8),
            ('2019-12-27',  9),
            ...
            ('2019-12-31', 14),
            ('2019-12-31', 15),
            ('2019-12-31', 16),
            ('2019-12-31', 17),
            ('2019-12-31', 18),
            ('2019-12-31', 19),
            ('2019-12-31', 20),
            ('2019-12-31', 21),
            ('2019-12-31', 22),
            ('2019-12-31', 23)],
           names=['orderdate', 'orderhour'], length=120)

In [180]:
unique_userid_in_1hour[('2019-12-27',  0)]

array([ 97707522, 170182475,  12532131, ..., 138483848, 132980262,
          923220])

In [187]:
for item in unique_userid_in_1hour[:5].items():
    print(item[0][1])
    print(item[1].shape[0])

0
1511
1
841
2
440
3
306
4
150


In [208]:
def calculate_concentrate_rate(order_per_hour, unique_buyers_per_hour):
    for item in order_per_hour.items():
        print(item)
#         concentrate_rate = order_per_hour[idx]#Number of Orders within 1 hour / Number of Unique Buyers within 1 hour
    return #concentrate_rate

calculate_concentrate_rate(df_groupby_orderhour, unique_userid_in_1hour)

AttributeError: 'DataFrameGroupBy' object has no attribute 'items'

In [222]:
group_shopid = df.groupby(['shopid'])
for name, group in group_shopid:
#     for user in group['userid']:
#         print(user)
    print(group['userid'].unique())

[196962305]
[ 2854032 48600461]
[168750452 194819216 130633421  62464559]
[  4401933  13837190  39828049  73993513  80643747  11753447 102616150
 162847440   8457753  96570515  99322339  81373632  12840239 115025463
  18193868  16021925 143378048  13999404  23352983  77184045  77391117
  90239949    996734  91048722 167932181 180772892  57738503   8838142
 177954889 151362536   6488128 121615825   1668819 151363170 132259411
 115450459  88321715  51586322  93206344 129074211 156270912 184840040
  68444473  33285757  22105044 181195859 136264815  86491592  27099291
 123715078  57956057 185802312  14062776 185601856]
[ 96020473  35220553 185421296  34507155 213907488   8149179  23867438
 136619968 112250289  10251073  12201862 131460045    495431   3827594
  99799957 106672631 191512878  37830887 138613531  27265170 166840413
  40405968 214839304  77424599 188459756   1700969 198765759  34632620
 137587441    546854 211937222  75514903 211192846  30534780  78632237
  89737965  96697084 1

[214781680  73439948  14828488 113115185 214776253]
[ 92192923  35781057 190355845 156702477 182873307  95456536 146709382
  27748642  11089665  37935357   2617982 101859093  93114799  89294190
    237014 183091859  46497903  21064545   1300599  97036577]
[141719243]
[183573973]
[77941861]
[63816183 11227013]
[ 70542895 138672449   1200890]
[112406022  44219114     27567 166027652 164477934  13219935]
[102220519]
[ 16684283 182506959 170960254 195195500]
[1273324]
[166137465  51332368  70617193]
[ 26254595 172676734]
[93242459]
[173285129    722632 121827333  76077674  76905043  46875705  10180682
  59115300 173197838  22390899  14576313    435308    610746]
[174170506   2059680]
[192395202]
[64018158]
[68938790 81815056]
[201940287 122151921]
[85866111]
[ 42990566 148392553  33750191  84152525]
[192897141]
[ 50800332 135140451  45930554 196183912  93448133 117006581  66898976
 100290135]
[121403248 142911136 209446109 214200060    490173 113958390 113379329
  59259911 176288758 160339

[186722585]
[184939397 163012690    182670  43012102 170850040 211151870  41132921
 102815472  15565051  18164816  66414833  63910003  71160670  74768187
  97403498 132324178 188215352 174207123  28080619 117677038]
[ 78274967  15428446  59686251   8521967 110638184 123497176   7437282
 111731490  95459468  34422422  11529818  81570759  31749415 101114993
 103580074  19986150 136701204   4194106 145355094 173944548 101161065
  92622776  65845991  15141807  20679063  43779623  36938131  23804697]
[167194009]
[2010733]
[176058612]
[98859620]
[207129074 144403143 180188116   9544957   5272171 132113554 152397749]
[155695715]
[70027944 69717958]
[118795475]
[244650]
[22219853]
[ 15396976  76247663  45836026  42792500  84950465  77912425  39802423
  22658728 107879819    677460 104876057  63924896 130795878 140014012
 197273179]
[ 31715411 105955664]
[ 11599968 166269321 208803326  85800661  60965521  66208846  43180886
  12088405 187075741 201594388 214927983  89307999 143768996 135724242


[ 71009029 185532707  72437498  85337925]
[112305334 133674101]
[132598104]
[12866739 80060998 21380471 58955788 19516539 43942200 23496239  1144746]
[ 53993995 133340279  23448727 213048065  83741693  10539858  62754982
  97900331  30109796 180498907 212641258 194690509 212742436 144928412
  31999708    214366  56493035 115255412 183379674 158475842   1122922
 101183150  21103345  44474002]
[214923033]
[168164885]
[124108460]
[ 72266748 136705021  15904547  11567869  33656824  42195173 204247258
  44820913 117421403  55737673  25927513 114988180  43921904  23804697
  50349485 126924077 187013024 187099276  26038152  71785794   1235249
   2101151  70412434 212495632 123492745 213682532  89576506   6632407
   6066181   2192844  81523593  39301585 123861555  99517519 127753883
  37417443   7006388  38432790  98568539 100689375  46871437 158139010
  71220574    309740 170993237   6662062  54471326 105304717  88361007
  33046849  65622949 158344631  78017109   5769280  56491210  42314861
 

  99348671 126471299  10161722]
[214350413 181067770]
[122592927 101183150]
[165333438 166280148 182358189]
[50514762]
[ 5314183 36443417]
[214642600]
[196423842  67596533]
[174534559]
[ 14621369 107268684   2106249 183437305 200455021 213071530 190090427
  61848586  46613676 107573520 171615636  88821168  43519195 110949249
 111321297  18432035]
[140098506]
[33153246]
[116592594  60560447  57721431]
[97204721]
[2199472]
[17918810]
[204742097]
[84757376]
[10925237]
[197922012]
[   726256 211355834 200912751 104069750 105218810 114400021]
[69213705]
[ 26805848  75877330 141014481 125404437  15130233   4915215  87613545
 135312802  38796709 116594015  88800227  29081401    740844 167406429
  92347511 122523434 178427449   2029696 214209607 108003911 195803320
 214628766 214692437   4770406 202532877  13292459  43936173  73883476
 214735943 128199958 174467130 100534957 168891049  80826877  97064487
 113110673  90335767  12524982 188572305 215309507  54228148  78280359
  51818858  6707004

...

[   236098 214958778]
[126502217 214590577]
[135342810]
[169953382 122835776   5916532 179329401]
[ 15948506   9308863  92466430  49296656 212761881  18308656  67300363
 173541980]
[160737281 155776474]
[136465606]
[ 31556279 166093334  70446221 213082080]
[193002894]
[177411357 120410560]
[199435095 208709766]
[38633826]
[ 46091479 174701151]
[157527130     12856  80070595 203817002]
[147853291]
[4924101]
[137313322  58205363 117047204 169645334 124630533 179702288]
[87028167]
[ 94689772 214508020]
[88366514]
[41689609]
[12594020]
[75343638]
[41455884]
[180387542]
[20553166]
[215234903]
[211348617    122242  19446530 178202778 189337238 134227883]
[38967424]
[19553967  1884330]
[88800596]
[145201725]
[  4345339  57115334 178614324 212936472  55776184 126502217  28494991]
[56233567]
[5802718]
[202541169]
[62281233 73537951]
[65814291]
[  1761705 157368989 156968165]
[125162017   5627531 117453023]
[181148927]
[22928902]
[206255177]
[97945812]
[158690055 204850996  48891070]
[92275977]


[129830336]
[5412085]
[189966910]
[167011073   2386582 177295709  75257056]
[83105138]
[198454044]
[139657662]
[210508776]
[101607989    549988  33462965]
[50198835]
[214909215]
[63120335]
[73269172 53580908]
[93554440]
[59073496]
[118758512]
[203619912]
[210508776]
[ 6185112 86974547]
[9491655  715642]
[174974844]
[158256272]
[75562537]
[197642661  50418974]
[121057357]
[ 17441967 134956485]
[110177954]
[179104225  97413321 121238188]
[159270743]
[12834236]
[159637368   6083430 155960588]
[80863157]
[193144402]
[70979013]
[197767082]
[214513011]
[ 1132517 36036000]
[116813529]
[214828897  80738641]
[81146964]
[177524261]
[65080828]
[61145855 53144838 33459354]
[83472178]
[102656257]
[48252298]
[195203337]
[24038395 10526332]
[194292345]
[7956892]
[89959915]
[213975472]
[197139705]
[96632374]
[7907298]
[158599726]
[214565483 138608935  60722510 203965446]
[169740521]
[73929421]
[169046011  99683761   1790216 201296641]
[103959955]
[135262216]
[72266748]
[168715081]
[214312226 215447850

[153882992  63563934]
[118401279]
[213157052]
[72081416]
[64276077  2691534]
[42172515]
[ 26931375 135315849 174640433   2223310]
[118142395  20067866 136724314]
[ 76488981 134379527]
[ 78099976 182243717]
[92341313]
[101452905]
[   233016 114282846]
[198436124  90149886]
[139317380]
[35126206]
[ 69806144 178412954 210989085]
[97220020]
[7074033]
[ 13854809  27368255  65452990 173986396  71590848]
[123865240]
[71556291]
[173314149]
[171640650 160087399 150436621]
[206940329]
[117410193]
[55605646]
[118429921]
[5537633]
[174613177   1479419 214493095   3444988]
[181802540]
[ 62826349  66746867 157862518 102933153 117358333   9358034 213595313
  54943300  89746874 111184598]
[214866456]
[107450999]
[105836742]
[208708129]
[201289968]
[100555378 214468577  65921305 163352656]
[57565]
[131926447]
[   282886 121403248]
[75643589]
[214123093  29879939 132481308   7143897 124925118]
[180504295  26523861  44225453 103571352  44167082  30805834  66435492
   1325103  99665654]
[117778099]
[ 7441

[26817820]
[47102577]
[13721631]
[192417131]
[156040805]
[174663144  62974681]
[43593597]
[20263793]
[115400698]
[101913633]
[179785954]
[151397528]
[  4219101  71366183   6195143 168807464 214179781  90077486 158788189]
[153613918]
[101489601]
[159430555 197732389]
[ 77666145 144925236  53479238]
[116765404]
[196032111]
[151177863 100789361 215200994]
[   765883 114288089 178639744  50803318 207324915  92424346 135891299
  88878185]
[ 59747066  94090661 184534317 172952277  93654136]
[215332292]
[187183520   2538885 110016388]
[ 58672436  25020624  75043525  73401688  95555289 215049712 203453922
 198107132]
[3140885]
[189147216 124483726  79642798 158851182 215003034]
[75848855]
[154010367 131713212  46281312  31059820 140304538]
[214323426]
[179331511]
[177746743 139494462]
[75887743]
[170236881 106548385    346742  83458419  51497439 102262212 198662175
 215368768 102385933 153162723]
[212578485 127264975  44167082]
[175275964]
[197093050]
[112029592  33322123  40390582  39308990]


[108228197]
[209852051  95536427 110225632]
[135291049]
[203966941  35514477]
[122067697]
[208506699]
[215101992]
[ 64563320 121031324]
[ 49630204 156891187]
[171410063]
[ 95810129 140900226]
[69349917 15626799 39257613]
[201174729]
[170304959]
[22380115]
[210544446]
[13450076]
[121228407]
[147853291]
[60863393]
[ 95842934    753950  78911819 172458174]
[60236601]
[215393529 139886670]
[20400631]
[111004389  42766291]
[158475842 202399143]
[210743380  80466843]
[54137522]
[87511534]
[87187746]
[151232380  29426301]
[124866308]
[125595125  57138742]
[69146041]
[90750423 21207141]
[168026397]
[67844833]
[79481714 15758586]
[884113]
[61381289]
[179122782]
[66858047]
[26494615]
[  3235354   3865864 121254702 158776840]
[61002066]
[214988128 114808783]
[71366183]
[132323650]
[211674712]
[50167978]
[383751]
[72961080]
[95399773]
[209831732]
[ 87964662  90077486  41024174  69311942 142650868  38163030 169046011
  56459985]
[185409691]
[6043096]
[39539977  8393902]
[205629102   3483931 1670176